In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

key = os.getenv("GEMINI_KEY")

In [3]:
#@title Setting up the Auth
import os
import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
import textwrap

genai.configure(api_key=key)

In [4]:
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [5]:
models = [m for m in genai.list_models()]
models

[Model(name='models/chat-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 Chat (Legacy)',
       description='A legacy text-only model optimized for chat conversations',
       input_token_limit=4096,
       output_token_limit=1024,
       supported_generation_methods=['generateMessage', 'countMessageTokens'],
       temperature=0.25,
       top_p=0.95,
       top_k=40),
 Model(name='models/text-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 (Legacy)',
       description='A legacy model that understands text and generates text as an output',
       input_token_limit=8196,
       output_token_limit=1024,
       supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
       temperature=0.7,
       top_p=0.95,
       top_k=40),
 Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
       display_name='Embedding Gecko',
       description='Obtai

In [6]:
model = genai.GenerativeModel("gemini-pro")
model

 genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
)

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.7,
                             google_api_key=key)

llm

ChatGoogleGenerativeAI(model='gemini-pro', google_api_key=SecretStr('**********'), client= genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
))

In [8]:
RESPONSE_JSON = """
{
    "1" : "question",
    "2" : "question",
    "3" : "question",
    ...
}

"""

In [9]:
TEMPLATE = """
You are an expert question generator. we are conducting an exam on \
{subject} your job is to generate 15 question of type {typ} for that subject.  \
Make sure the questions are not repeated and check all the questions to be conforming the text as well. \
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} Questions \
### RESPONSE_JSON  \ 
{response_json} \

"""

In [10]:
number = 15
subject = "financial reporting"
typ = "Open Quetion"

In [11]:
gen_prompt = ChatPromptTemplate.from_template(TEMPLATE)

output_parser = StrOutputParser()

In [12]:
generation = gen_prompt | llm | output_parser

In [13]:
quiz = generation.invoke({
    "subject" : subject,
    "typ" : typ,
    "number" : number,
    "response_json" : RESPONSE_JSON
})

# to_markdown(response)

In [14]:
print(quiz)

### RESPONSE_JSON

{
    "1": "Explain the concept of materiality in financial reporting and its significance in determining which items should be disclosed in the financial statements.",
    "2": "Describe the different financial statement formats used for different types of entities and explain the reasons for these differences.",
    "3": "Discuss the accounting treatment of contingent liabilities and the potential implications for financial reporting.",
    "4": "Explain the importance of notes to financial statements and provide examples of the types of information typically disclosed in these notes.",
    "5": "Describe the objectives of the conceptual framework for financial reporting and how it guides the development of accounting standards.",
    "6": "Discuss the qualitative characteristics of financial information and their importance in assessing the usefulness of financial statements.",
    "7": "Explain the role of the auditor in the financial reporting process and descri

In [15]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Question for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz:
{quiz}

Check from an expert English Writer of the above quiz:

Generate the response in the following format:
### RESPONSE_JSON
{response_json}
"""

eval_prompt = ChatPromptTemplate.from_template(TEMPLATE2)

In [16]:
evaluation = eval_prompt | llm | output_parser

In [17]:
quiz_eval = evaluation.invoke({
    "subject" : subject,
    "quiz" : quiz,
    "response_json" : RESPONSE_JSON
})

print(quiz_eval)

**Complexity Analysis:**

The quiz questions are comprehensive and cover a wide range of financial reporting topics. They require students to demonstrate a deep understanding of accounting concepts, principles, and standards. The questions are appropriate for students with a solid foundation in financial reporting.

**Updated Quiz Questions:**

No changes are necessary as the quiz questions are well-structured and aligned with the cognitive and analytical abilities of financial reporting students.

### RESPONSE_JSON

{
    "1": "Explain the concept of materiality in financial reporting and discuss its significance in determining which items should be disclosed in the financial statements.",
    "2": "Describe the different financial statement formats used for various types of entities and explain the reasons for these differences.",
    "3": "Discuss the accounting treatment of contingent liabilities and their potential implications for financial reporting.",
    "4": "Explain the impo

In [18]:
import re

# Regular expression pattern
pattern = r'\{.*?\}'

# Extracting substring
substring = re.search(pattern, quiz, re.DOTALL)

if substring:
    substring = substring.group()
    print(substring)
else:
    print("No response_json found.")


{
    "1": "Explain the concept of materiality in financial reporting and its significance in determining which items should be disclosed in the financial statements.",
    "2": "Describe the different financial statement formats used for different types of entities and explain the reasons for these differences.",
    "3": "Discuss the accounting treatment of contingent liabilities and the potential implications for financial reporting.",
    "4": "Explain the importance of notes to financial statements and provide examples of the types of information typically disclosed in these notes.",
    "5": "Describe the objectives of the conceptual framework for financial reporting and how it guides the development of accounting standards.",
    "6": "Discuss the qualitative characteristics of financial information and their importance in assessing the usefulness of financial statements.",
    "7": "Explain the role of the auditor in the financial reporting process and describe the key audit pr

In [19]:
import json

quiz_json = json.loads(substring)

In [20]:
import pandas as pd

df = pd.DataFrame.from_dict(quiz_json, orient = "index", columns=["Question"])
df.head()

,Question
1,Explain the concept of materiality in financia...
2,Describe the different financial statement for...
3,Discuss the accounting treatment of contingent...
4,Explain the importance of notes to financial s...
5,Describe the objectives of the conceptual fram...


In [21]:
print(df['Question'][1])

Describe the different financial statement formats used for different types of entities and explain the reasons for these differences.


In [22]:
df.to_csv("Questions.csv")

In [23]:
TEMPLATE3 = """
Hey, I need your help reviewing my answer for a question.
grade can be any floating point number

Here's the question: 
{que}

My answer: 
{ans}

Generate the response in the following format:
### Grade 
grade/10
"""

grade_prompt = ChatPromptTemplate.from_template(TEMPLATE3)

In [24]:
grading = grade_prompt | llm | output_parser

In [25]:
que = df['Question'][1]
ans = "Materiality in financial reporting refers to the importance or significance of information to the users of financial statements. Information is considered material if its omission or misstatement could influence the economic decisions of users. The concept of materiality is subjective and depends on factors such as the size, nature, and circumstances of the item. Significance in financial reporting ensures that only relevant information is included in the financial statements, preventing them from being cluttered with trivial details. Materiality guides accountants and auditors in determining what information to disclose and what to omit, helping users focus on key aspects that impact decision-making."

In [26]:
grade = grading.invoke({
    "que" : que,
    "ans" : ans
})

print(grade)

### Grade
2/10


In [27]:
search_word = "Grade\n"

# Find the index where RESPONSE_JSON occurs
index = quiz.find(search_word)

# Extract the substring after RESPONSE_JSON
if index != -1:
    substring = quiz[index + len(search_word):]
    print(substring)
else:
    print("RESPONSE_JSON not found in the string.")


RESPONSE_JSON not found in the string.


In [28]:
import re

In [29]:
pattern = r'\b\d+\b'

match = re.search(pattern, grade)

if match:
    # Extract the integer grade
    grade = int(match.group())
    print("Grade:", grade)
else:
    print("No grade found.")

Grade: 2


In [30]:
def review_grade(row):
    grade = grading.invoke({
    "que" : row['Question'],
    "ans" : row['Answer']
    })

    pattern = r'\b\d+\b'

    match = re.search(pattern, grade)

    if match:
        # Extract the integer grade
        grade = int(match.group())
        print("Grade:", grade)
    else:
        print("No grade found.")
    
    return grade

In [31]:
def generate_answer(que):
    return model.generate_content(que).text

In [32]:
result = pd.read_csv("Questions.csv")

result

,Unnamed: 0,Question
0,1,Explain the concept of materiality in financia...
1,2,Describe the different financial statement for...
2,3,Discuss the accounting treatment of contingent...
3,4,Explain the importance of notes to financial s...
4,5,Describe the objectives of the conceptual fram...
5,6,Discuss the qualitative characteristics of fin...
6,7,Explain the role of the auditor in the financi...
7,8,Describe the different types of financial rati...
8,9,Discuss the ethical considerations that accoun...
9,10,Explain the concept of consolidated financial ...


In [36]:
answers = []

for question in result['Question']:
    answers.append(generate_answer(question))

answers

["**Concept of Materiality in Financial Reporting**\n\nMateriality is a fundamental concept in financial reporting that refers to the significance of financial information that should be disclosed in the financial statements. It determines whether an item must be disclosed to avoid misleading users of the financial statements.\n\n**Significance of Materiality**\n\nMateriality ensures that only information that has a significant impact on the financial position, results of operations, or cash flows of a company is included in the financial statements. It helps prevent clutter and focuses attention on the most important aspects of a company's financial performance.\n\n**Determining Materiality**\n\nDetermining materiality is a judgmental process that considers both qualitative and quantitative factors:\n\n**Qualitative Factors:**\n\n* Nature of the item\n* Potential impact on decision-making\n* Sensitivity of users of the financial statements\n\n**Quantitative Factors:**\n\n* Size or mag

In [38]:
result['Answer'] = answers
result

,Unnamed: 0,Question,Answer
0,1,Explain the concept of materiality in financia...,**Concept of Materiality in Financial Reportin...
1,2,Describe the different financial statement for...,**Different Financial Statement Formats**\n\nT...
2,3,Discuss the accounting treatment of contingent...,**Accounting Treatment of Contingent Liabiliti...
3,4,Explain the importance of notes to financial s...,**Importance of Notes to Financial Statements*...
4,5,Describe the objectives of the conceptual fram...,**Objectives of the Conceptual Framework for F...
5,6,Discuss the qualitative characteristics of fin...,**Qualitative Characteristics of Financial Inf...
6,7,Explain the role of the auditor in the financi...,**Role of the Auditor in the Financial Reporti...
7,8,Describe the different types of financial rati...,**Types of Financial Ratios**\n\n**1. Liquidit...
8,9,Discuss the ethical considerations that accoun...,**Ethical Considerations in Financial Reportin...
9,10,Explain the concept of consolidated financial ...,**Concept of Consolidated Financial Statements...


In [ ]:
# result['Answer'] = result['Question'].apply(generate_answer)
# result['Answer']

In [39]:
result['Answer'][0]

"**Concept of Materiality in Financial Reporting**\n\nMateriality is a fundamental concept in financial reporting that refers to the significance of financial information that should be disclosed in the financial statements. It determines whether an item must be disclosed to avoid misleading users of the financial statements.\n\n**Significance of Materiality**\n\nMateriality ensures that only information that has a significant impact on the financial position, results of operations, or cash flows of a company is included in the financial statements. It helps prevent clutter and focuses attention on the most important aspects of a company's financial performance.\n\n**Determining Materiality**\n\nDetermining materiality is a judgmental process that considers both qualitative and quantitative factors:\n\n**Qualitative Factors:**\n\n* Nature of the item\n* Potential impact on decision-making\n* Sensitivity of users of the financial statements\n\n**Quantitative Factors:**\n\n* Size or magn